# D13 Số liệu dạng chuỗi kí tự

## Mục đích

Trong bài này chúng ta sẽ làm quen với các thao tác xử lí chuỗi kí tự trong Pandas.


## Tìm kiếm

Kịch bản đầu tiên chúng ta sẽ tìm hiểu là một kịch bản đơn giản hóa của Google Forms. Khi bạn tạo những câu hỏi cho phép chọn nhiều phương án ở trên Google Forms, đáp án mà bạn xuất ra sẽ bị "dính" vào với nhau như trong ví dụ dưới đây.

In [1]:
import pandas as pd

d1 = pd.DataFrame({
    "id": range(1, 6),
    "nghe_yeuthich": [
        "Bác sĩ",
        "Bác sĩ;Điều dưỡng;Công nhân",
        "Bác sĩ;Công nhân",
        "Điều dưỡng;Công nhân;Khác",
        "Công nhân;Khác"
    ]
})

d1

,id,nghe_yeuthich
0,1,Bác sĩ
1,2,Bác sĩ;Điều dưỡng;Công nhân
2,3,Bác sĩ;Công nhân
3,4,Điều dưỡng;Công nhân;Khác
4,5,Công nhân;Khác


Bây giờ nếu chúng ta muốn xem những ai lựa chọn đáp án Công nhân, chúng ta sẽ cần tìm xem chuỗi kí tự "Công nhân" có tồn tại trong các câu trả lời hay không. Chúng ta sẽ sử dụng hàm `contains()`. Hàm này được truy cập thông qua accessor `str` (sử dụng cho tất cả các hàm thao tác trên chuỗi kí tự). Xem ví dụ dưới đây.

In [2]:
d1["nghe_yeuthich"].str.contains("Công nhân")

0    False
1     True
2     True
3     True
4     True
Name: nghe_yeuthich, dtype: bool

Trong trường hợp chúng ta muốn kiểm tra tương tự cho tất cả các phương án thì phải làm thế nào? Bạn có thể sử dụng vòng lặp `for`.

In [3]:
choices = ["Bác sĩ", "Điều dưỡng", "Công nhân", "Khác"]

d2 = d1.copy(deep=True)

for choice in choices:
    d2[choice] = d2["nghe_yeuthich"].str.contains(choice)

d2

,id,nghe_yeuthich,Bác sĩ,Điều dưỡng,Công nhân,Khác
0,1,Bác sĩ,True,False,False,False
1,2,Bác sĩ;Điều dưỡng;Công nhân,True,True,True,False
2,3,Bác sĩ;Công nhân,True,False,True,False
3,4,Điều dưỡng;Công nhân;Khác,False,True,True,True
4,5,Công nhân;Khác,False,False,True,True


Bạn cũng có thể sử dụng hàm `map()`. Nhớ rằng hàm `map()` trả về một iterator (trong ví dụ mỗi item của iterator là một Pandas series), và để ghép được các series theo cột (dùng tính năng concat), chúng ta phải chuyển nó thành dạng danh sách trước. Nhìn vào hàm lambda, bạn có thể thấy mình sử dụng thêm hàm `rename()` để đổi tên series thành tên của phương án; nếu không đổi tên, series giữ nguyên tên gốc là `"nghe_yeuthich"`. Sau khi tạo ra các cột mới, chúng ta join data frame bổ sung với bộ số liệu gốc.

In [4]:
d1.join(
    pd.concat(
        list(map(lambda x: d1["nghe_yeuthich"].str.contains(x) \
            .rename(x), choices)
        ), axis=1
    )
)

,id,nghe_yeuthich,Bác sĩ,Điều dưỡng,Công nhân,Khác
0,1,Bác sĩ,True,False,False,False
1,2,Bác sĩ;Điều dưỡng;Công nhân,True,True,True,False
2,3,Bác sĩ;Công nhân,True,False,True,False
3,4,Điều dưỡng;Công nhân;Khác,False,True,True,True
4,5,Công nhân;Khác,False,False,True,True


Trong trường hợp bạn không muốn sử dụng nội dung của các phương án làm tên cột, bạn có thể sử dụng một từ điển để đổi tên. Ví dụ:

In [5]:
datadict = {
    "Bác sĩ": "01_bacsi",
    "Điều dưỡng": "02_dieuduong",
    "Công nhân": "03_congnhan",
    "Khác": "09_khac"
}

d1.join(
    pd.concat(
        list(map(lambda x: d1["nghe_yeuthich"].str.contains(x) \
            .rename(f"nghe_yeuthich__{datadict[x]}"), choices)
        ), axis=1
    )
)

,id,nghe_yeuthich,nghe_yeuthich__01_bacsi,nghe_yeuthich__02_dieuduong,nghe_yeuthich__03_congnhan,nghe_yeuthich__09_khac
0,1,Bác sĩ,True,False,False,False
1,2,Bác sĩ;Điều dưỡng;Công nhân,True,True,True,False
2,3,Bác sĩ;Công nhân,True,False,True,False
3,4,Điều dưỡng;Công nhân;Khác,False,True,True,True
4,5,Công nhân;Khác,False,False,True,True


Ngoài hàm `contains()`, bạn cũng có thể quan tâm tới hàm `startswith()` và `endswith()` trong trường hợp cần phát hiện các chuỗi kí tự mở đầu và kết thúc.


## Tách các chuỗi kí tự

Với bài toán trên, chúng ta có một giải pháp khác loằng ngoằng hơn một chút. Đầu tiên chúng ta nhận thấy các phương án được phân cách bằng dấu chấm phẩy `";"`. Chúng ta có thể dùng nó để tách các chuỗi kí tự của các phương án.

In [6]:
d2 = d1.copy(deep=True)
d2 = d2.assign(nghe_yeuthich = d2["nghe_yeuthich"].str.split(";"))
d2

,id,nghe_yeuthich
0,1,[Bác sĩ]
1,2,"[Bác sĩ, Điều dưỡng, Công nhân]"
2,3,"[Bác sĩ, Công nhân]"
3,4,"[Điều dưỡng, Công nhân, Khác]"
4,5,"[Công nhân, Khác]"


Trong đoạn lệnh trên, thay vì phép gán thông thường để tạo cột mới hoặc ghi đè dữ liệu lên cột cũ, mình sử dụng hàm `assign()`. Hai cách làm này tương tự nhau, ưu điểm là bạn có thể chain các hàm mà không bị đứt đoạn vì các lệnh tạo cột mới. Cách làm này tương tự như hàm `dplyr::mutate()` trong R.

Kết quả trả về là một series, mỗi phần tử là một danh sách. Pandas hỗ trợ hàm `explode()` để giải phóng danh sách này thành các hàng. Bạn nên explode một data frame vì khi explode series, Pandas sẽ không giữ lại index của series ban đầu.

In [7]:
d2 = d2.explode("nghe_yeuthich")
d2

,id,nghe_yeuthich
0,1,Bác sĩ
1,2,Bác sĩ
1,2,Điều dưỡng
1,2,Công nhân
2,3,Bác sĩ
2,3,Công nhân
3,4,Điều dưỡng
3,4,Công nhân
3,4,Khác
4,5,Công nhân


Lúc này bạn có thể dùng hàm `pivot()` để chuyển các hàng này thành các cột.

In [8]:
d2["value"] = 1
d2.pivot(index="id", columns="nghe_yeuthich", values="value")

nghe_yeuthich,Bác sĩ,Công nhân,Khác,Điều dưỡng
id,,,,
1,1.0,NaN,NaN,NaN
2,1.0,1.0,NaN,1.0
3,1.0,1.0,NaN,NaN
4,NaN,1.0,1.0,1.0
5,NaN,1.0,1.0,NaN


Chúng ta sẽ dùng hàm `fillna()` để lấp đầy các giá trị NA tạo ra sau khi pivot.

In [9]:
d2.pivot(index="id", columns="nghe_yeuthich", values="value") \
    .fillna(0, downcast="infer")

nghe_yeuthich,Bác sĩ,Công nhân,Khác,Điều dưỡng
id,,,,
1,1,0,0,0
2,1,1,0,1
3,1,1,0,0
4,0,1,1,1
5,0,1,1,0


Bạn cũng có thể đổi tên các cột mới tạo.

In [10]:
d2.pivot(index="id", columns="nghe_yeuthich", values="value") \
    .fillna(0, downcast="infer") \
    .rename(columns=lambda x: f"nghe_yeuthich__{datadict[x]}")

nghe_yeuthich,nghe_yeuthich__01_bacsi,nghe_yeuthich__03_congnhan,nghe_yeuthich__09_khac,nghe_yeuthich__02_dieuduong
id,,,,
1,1,0,0,0
2,1,1,0,1
3,1,1,0,0
4,0,1,1,1
5,0,1,1,0


May mắn thay, Pandas cho chúng ta hàm `get_dummies()` để không phải thực hiện công việc trên lằng nhằng như vậy. Nếu các phương án của bạn được phân cách bởi một kí tự hoặc chuỗi kí tự đặc biệt, không xuất hiện trong nội dung các phương án, bạn có thể lựa chọn phương thức tiện lợi này (thiết lập kí tự phân cách bằng đối số `sep`).

In [11]:
d1["nghe_yeuthich"].str.get_dummies(sep=";")

,Bác sĩ,Công nhân,Khác,Điều dưỡng
0,1,0,0,0
1,1,1,0,1
2,1,1,0,0
3,0,1,1,1
4,0,1,1,0


## Thay thế chuỗi kí tự

Đôi khi bạn cũng sẽ cần thay thế chuỗi kí tự. Chẳng hạn, trong ví dụ dưới đây, chúng ta có hai cột `"mean"` và `"sd"` tạo ra từ việc aggregate số liệu. Bạn muốn tạo ra một chuỗi kí tự để thể hiện Mean (SD) và chuyển dấu phân cách số thập phân từ dấu chấm thành dấu phẩy.

In [12]:
d3 = pd.DataFrame({
    "subgroup": [1, 2, 3, 4],
    "mean": [2.30, 4.35, 1.66, 4.29],
    "sd": [1.12, 1.16, 1.67, 0.89]
})

d3 = d3.assign(mean_sd=d3[["mean", "sd"]].apply(lambda x: "{:.2f} ({:.2f})".format(*x), axis=1))
d3

,subgroup,mean,sd,mean_sd
0,1,2.30,1.12,2.30 (1.12)
1,2,4.35,1.16,4.35 (1.16)
2,3,1.66,1.67,1.66 (1.67)
3,4,4.29,0.89,4.29 (0.89)


Bạn có thể dùng hàm `replace()`. Hàm này cho phép bạn thay thế một chuỗi kí tự con trong toàn bộ chuỗi kí tự bằng một chuỗi mới; chuỗi kí tự có thể là một chuỗi cố định hoặc ở dạng pattern đại diện cho nhiều chuỗi kí tự khác nhau. Với cách thứ hai, chúng ta sẽ sử dụng regular expression (RegEx), và chúng ta sẽ có một phần dành riêng cho RegEx.

In [13]:
d3["mean_sd"].str.replace(".", ",", regex=False)

0    2,30 (1,12)
1    4,35 (1,16)
2    1,66 (1,67)
3    4,29 (0,89)
Name: mean_sd, dtype: object

Trong trường hợp muốn kết hợp vào trong hàm `assign()` ở trên, bạn có thể làm như sau:

In [14]:
d3 = d3.assign(
    mean_sd=d3[["mean", "sd"]].apply(lambda x: "{:.2f} ({:.2f})".format(*x), axis=1) \
        .str.replace(".", ",", regex=False)
)
d3

,subgroup,mean,sd,mean_sd
0,1,2.30,1.12,"2,30 (1,12)"
1,2,4.35,1.16,"4,35 (1,16)"
2,3,1.66,1.67,"1,66 (1,67)"
3,4,4.29,0.89,"4,29 (0,89)"


Bạn có thể xem thêm danh sách các hàm xử lí chuỗi kí tự ở [đây](https://pandas.pydata.org/docs/reference/series.html#api-series-str).

---

[Bài trước](./12_merge.ipynb) - [Danh sách bài](../README.md) - [Bài sau]()